# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235225402315                   -0.50    7.0
  2   -7.250340532478       -1.82       -1.40    1.0
  3   -7.251307603751       -3.01       -2.13    3.0
  4   -7.251275431621   +   -4.49       -2.22    4.0
  5   -7.251329479446       -4.27       -2.68    3.0
  6   -7.251337454158       -5.10       -3.06    2.0
  7   -7.251338761756       -5.88       -3.90    1.0
  8   -7.251338793249       -7.50       -4.17    4.0
  9   -7.251338798239       -8.30       -4.68    2.0
 10   -7.251338798639       -9.40       -5.15    3.0
 11   -7.251338798702      -10.20       -5.89    2.0
 12   -7.251338798704      -11.64       -6.42    3.0
 13   -7.251338798705      -12.92       -6.77    3.0
 14   -7.251338798705      -13.53       -7.33    2.0
 15   -7.251338798705      -15.05       -7.71    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07479864810358919
[ Info: Arnoldi iteration step 2: normres = 0.5409380580835781
[ Info: Arnoldi iteration step 3: normres = 0.6225725773979943
[ Info: Arnoldi iteration step 4: normres = 0.29953018258155645
[ Info: Arnoldi iteration step 5: normres = 0.6055641023706135
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.44e-02, 8.74e-02, 5.17e-01, 3.01e-01, 2.05e-02)
[ Info: Arnoldi iteration step 6: normres = 0.14711409350535606
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.96e-03, 8.41e-02, 8.38e-02, 6.88e-02, 1.33e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09577188581853702
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.27e-04, 6.23e-03, 6.37e-03, 2.44e-02, 8.69e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11501035291511016
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.27e